In [ ]:
import pandas as pd
import numpy as np

perf_amazon_reviews_multi_en_fine = "nlp_stitching-amazon_reviews_multi_en_fine_grained.tsv"
perf_amazon_reviews_multi_en_coarse = "nlp_stitching-amazon_reviews_multi_en_coarse.tsv"

perf_dbpedia_14 = "nlp_stitching-dbpedia_14.tsv"
perf_trec = "nlp_stitching-trec.tsv"


trec = pd.read_csv(perf_trec, sep="\t", index_col=0)
trec

dbpedia = pd.read_csv(perf_dbpedia_14, sep="\t", index_col=0)
dbpedia

amazon_coarse = pd.read_csv(perf_amazon_reviews_multi_en_coarse, sep="\t", index_col=0)
amazon_coarse

amazon_fine = pd.read_csv(perf_amazon_reviews_multi_en_fine, sep="\t", index_col=0)
amazon_fine

In [ ]:
def display_results(df, drop_transformers=[]):
    for transformer_to_drop in drop_transformers:
        df = df[df.embed_transformer != transformer_to_drop]
        df = df[df.classifier_transformer != transformer_to_drop]
    o = (
        df.drop(columns=["seed", "precision", "recall", "embed_transformer", "classifier_transformer"])
        .groupby(["embed_type", "stitched"])
        .agg([np.mean, np.std, "count"])
    )
    print(o)
    return o

In [ ]:
trec_df = display_results(trec, drop_transformers=["xlm-roberta-base"])

In [ ]:
dbpedia_df = display_results(dbpedia, drop_transformers=["xlm-roberta-base"])

In [ ]:
amazon_coarse_df = display_results(amazon_coarse, drop_transformers=["xlm-roberta-base"])

In [ ]:
amazon_fine_df = display_results(amazon_fine, drop_transformers=["xlm-roberta-base"])

In [ ]:
def latex_float(f):
    float_str = "{0:.2f}".format(f)
    if "e" in float_str:
        base, exponent = float_str.split("e")
        return r"{0} \times 10^{{{1}}}".format(base, int(exponent))
    else:
        return float_str


def extract_mean_std(df: pd.DataFrame, model_type: str, stitching: bool) -> str:
    df = df * 100
    df = df.round(2)
    try:
        mean_std = df.loc[model_type, stitching]
        return rf"${latex_float(mean_std['fscore']['mean'])} \pm {latex_float(mean_std['fscore']['std'])}$"
    except (AttributeError, KeyError):
        return "?"


classification_rel = r"{} & {} & {} & {} & {} \\[1ex]"

for available_model_type in ("absolute", "relative"):
    for stitching, stit_name in zip(
        [
            False,
            True,
        ],
        [
            "Non-Stitch",
            "Stitch",
        ],
    ):

        s = classification_rel.format(
            available_model_type,
            stit_name,
            *[
                extract_mean_std(df, available_model_type, stitching)
                for df in [trec_df, dbpedia_df, amazon_coarse_df, amazon_fine]
            ],
        )
        print(s)